In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install torch torchvision torchaudio
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 1.2 MB/s eta 0:00:00


In [ ]:
import networkx as nx
import matplotlib.pyplot as plt

# HELPER FUNCTIONS

In [ ]:
import networkx as nx
import random
import numpy
from torch_geometric.data import Data


def mc_upper_bound(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "chromatic_number" integer upper bound on the maximum clique number
	"""
	answ = nx.algorithms.coloring.greedy_color(G)
	chromatic_number = list(set(list(answ.values())))
	return len(chromatic_number)

def mc_lower_bound(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "lower bound" list of variables which form a clique in G
	"""
	return nx.maximal_independent_set(nx.complement(G))

def edge_k_core(G, k):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	 - "k" Integer that is at least one less than the global maximum clique number
	OUTPUT:
	 - "G" Networkx Undirected Graph where edge k-core reduction has been applied
	"""
	for a in list(G.edges()):
		x = list(G.neighbors(a[0]))
		y = list(G.neighbors(a[1]))
		if len(list(set(x) & set(y))) <= (k-2):
			G.remove_edge(a[0], a[1])
	return G

def k_core_reduction(graph, k):
	"""
	INPUT:
	 - "graph" Networkx Undirected Graph
	 - "k" Integer that is at least one less than the global maximum clique number
	OUTPUT:
	 - "graph" Networkx Undirected Graph where k-core reduction has been applied
	"""
	graph = nx.k_core(graph, k)
	ref1 = len(list(graph.edges()))
	graph = edge_k_core(graph, k)
	ref2 = len(list(graph.edges()))
	while ref1 != ref2:
		if len(graph) == 0:
			return graph
		graph = nx.k_core(graph, k)
		ref1 = len(list(graph.edges()))
		graph = edge_k_core(graph, k)
		ref2 = len(list(graph.edges()))
	return graph

def is_clique(G):
	"""
	INPUT:
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "True" if G is a clique, and "False" if G is not a clique
	"""
	n = len(list(G.nodes()))
	m = len(list(G.edges()))
	if int(m) == int((n*(n-1))/float(2)):
		return True
	else:
		return False

def ch_partitioning(vertex, G):
	"""
	INPUT:
	 - "vertex" splitting vertex
	 - "G" Networkx Undirected Graph
	OUTPUT:
	 - "SSG" Left subgraph after partitioning
	 - "SG" Right subgraph after partitioning
	"""
	n = list(G.neighbors(vertex))
	Gp = []
	for iter in list(G.edges()):
		if iter[0] in n:
			if iter[1] in n:
				Gp.append(iter)
	G.remove_node(vertex)
	return nx.Graph(Gp), G

def lowest_degree_vertex(graph):
	"""
	INPUT:
	 - "graph" Networkx Undirected Graph
	OUTPUT:
	 - "i" node that has the lowest degree in the graph
	"""
	degrees = [graph.degree(a) for a in list(graph.nodes())]
	minimum = min(degrees)
	for i in list(graph.nodes()):
		if graph.degree(i) == minimum:
			return i

def highest_degree_vertex(graph):
	"""
	INPUT:
	 - "graph" Networkx Undirected Graph
	OUTPUT:
	 - "i" node that has the lowest degree in the graph
	"""
	degrees = [graph.degree(a) for a in list(graph.nodes())]
	maximum = max(degrees)
	for i in list(graph.nodes()):
		if graph.degree(i) == maximum:
			return i

def remove_zero_degree_nodes(graph):
    """
    INPUT:
    - "graph" Networkx Undirected Graph
    OUTPUT:
    - "graph" Networkx Undirected Graph with no zero degree nodes
    """
    nodes = list(graph.nodes())
    for n in nodes:
      if graph.degree(n) == 0:
        graph.remove_node(n)
    return graph

# average of |G| - |SG| and |G| - |SSG|
def average_node_decrease(original_graph_size, subgraph1_size, subgraph2_size, bound_improve):
  if subgraph1_size == 0 and subgraph2_size == 0:
    return original_graph_size
  elif subgraph1_size == 0:
    return original_graph_size - subgraph2_size
  elif subgraph2_size == 0:
    return original_graph_size - subgraph1_size
  else:
    return (2*original_graph_size - subgraph2_size - subgraph1_size)/2

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import matplotlib.pyplot as plt
from torch_geometric.nn import GCNConv
import torch.nn.functional as F

In [ ]:
def ch_partitioning_eval(G, LIMIT, eval_function):
  k = mc_lower_bound(G)
  og_bound = len(k)
  graph = k_core_reduction(G, len(k))
  og_size = len(G)
  eval_list = []
  node_eval_list = []
  best_eval = -10000
  best_node = 0
  for vertex in tqdm(list(G.nodes())):
    graph = G.copy()
    vertex_removals = {graph: []}
    SG = graph
    subgraphs = [graph]
    vcount = vertex_removals[SG]
    del vertex_removals[SG]
    SSG, SG = ch_partitioning(vertex, graph)
    SG = remove_zero_degree_nodes(SG)
    SSG = remove_zero_degree_nodes(SSG)
    SG = k_core_reduction(SG, len(k)-len(vcount)) #0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) #1
    vertex_removals[SSG] = vcount+[vertex]
    vertex_removals[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removals[SSG])) == True
      if len(SSG)+len(vertex_removals[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removals[SSG]
      del vertex_removals[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removals[SG])) == True
      if len(SG)+len(vertex_removals[SG]) > len(k):
        k = list(SG.nodes())+vertex_removals[SG]
      del vertex_removals[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removals[SSG]
      #print(vertex_removals[SSG])
      assert is_clique(G.subgraph(SSG_lower)) == True
      if len(SSG_lower) > len(k):
        vcount = vertex_removals[SSG]
        del vertex_removals[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removals[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removals[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            del vertex_removals[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removals[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removals.keys()):
        sub_solution_SSG = vertex_removals[SSG]
        del vertex_removals[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removals[SG]
      #print(vertex_removals[SG])
      assert is_clique(G.subgraph(SG_lower)) == True
      if len(SG_lower) > len(k):
        vcount = vertex_removals[SG]
        del vertex_removals[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removals[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removals[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            del vertex_removals[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removals[SG]
    if len(SG) == 0:
      if SG in list(vertex_removals.keys()):
        sub_solution_SG = vertex_removals[SG]
        del vertex_removals[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
    # define eval parameters
    SG_size =  len(SG)
    if SG_size <= LIMIT:
      SG_size = 0
    SSG_size = len(SSG)
    if SSG_size <= LIMIT:
      SSG_size = 0
    bound_improvement = len(k) - og_bound
    current_eval = eval_function(og_size, SG_size, SSG_size, bound_improvement)
    eval_list.append(current_eval)
    node_eval_list.append(vertex)
    if current_eval > best_eval:
      best_eval = current_eval
      best_node = vertex
  temp = [[x, y] for x, y in zip(node_eval_list, eval_list)]
  node_eval_list = temp
  return node_eval_list, best_eval, best_node

In [ ]:
# Generating Labels Function
def label_generation(graph, LIMIT=40, eval_function=average_node_decrease):
  """
  Input: networkx graph, decomposition LIMIT
  - Preprocess graph (kcore)
  - Assign labels to each node
  Output: networkx graph with node labels
  """
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting Labelling Algorithm ===")
  G = graph.copy()
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    return
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  vertex_removal = {graph: []}
  subgraphs = [graph]
  SG = subgraphs.pop()
  SG = remove_zero_degree_nodes(SG)
  assert len(SG) != 0
  vcount = vertex_removal[SG]
  del vertex_removal[SG]
  print("Computing Labels")
  SG_copy = SG.copy()
  eval_list,_,_ = ch_partitioning_eval(SG_copy, LIMIT, eval_function)
  for node_eval in eval_list:
    node = node_eval[0]
    eval = node_eval[1]
    G.nodes[node]['label'] = eval
  return G

def max_normalize(label_list, k):
  # normalize data
  max_val = max(label_list)
  normalized_data = [(x / max_val) for x in label_list]
  return normalized_data


# Normalize dataset function
def dataset_normalize(dataset, normalize_function=max_normalize, normalize_param=0.2):
  for graph_idx in tqdm(range(len(dataset))):
    data = dataset[graph_idx]
    node_labels_list = data.y.tolist()
    normalized_node_labels_list = normalize_function(node_labels_list, normalize_param)
    data.y = torch.tensor(normalized_node_labels_list)
  return dataset



In [ ]:

def DBK_model_instance(pygraph, model, LIMIT=40):
  """
  INPUT:
    - "graph" must be a Networkx Undirected Graph
    - "LIMIT" is an integer describing the largest size of graph which solver_func can solve; all subgraph sizes solved will be less than or equal to LIMIT
    - "solver_function" takes a Networkx Graph, and outputs a list of nodes which are hopefully the Maximum Clique elements; it can be an approximate or exact solver function
  OUTPUT:
    - "k" is a list of graph nodes which form a clique in the input graph. If the solver is exact, then k is the Maximum Clique
  NOTES:
    - The central idea of using bounds is that we maintain a global lower bound on the Maximum Clique. Then, for each sub problem we calculate a fast upper bound.
      If any sub problem has an upper bound which is less than or equal to the global lower bound, we can remove that sub problem from consideration in the remaining iterations of the algorithm.
    - This algorithm does not necessarily enumerate all cliques nor all Maximum Cliques. In particular, it is designed to return a single maximum clique assuming the solver is exact.
      However, the algorithm could be modified to include all maximum cliques found from solving each sub-problem.
    - There are many assert statements in this function. These all serve as "sanity checks"; if any of them are tripped, something went wrong or an input was incorrect
  """
  start_time = time.time()
  device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
  model.to(device)
  print("Model's device:", next(model.parameters()).device)
  graph = to_networkx(pygraph)
  graph.remove_edges_from(nx.selfloop_edges(graph))
  # TRAIN THE MODEL
  labeled_graph = label_generation(graph)
  labeled_pygraph = graph_to_pyg_data(labeled_graph)
  labeled_pygraph.x = get_features(labeled_graph)
  labeled_pygraph.x = labeled_pygraph.x.to(torch.float32)
  normalized_pygraph = dataset_normalize([labeled_pygraph])[0]
  criterion = nn.MSELoss()
  optimizer = optim.Adam(model.parameters(), lr=0.0001)
  # Train the model with validation
  normalized_pygraph.to(device)
  train_model_regression(model, [normalized_pygraph], [normalized_pygraph], optimizer, criterion, batch_size=1, patience=100, epochs=2000)
  model.eval()
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting DBK Algorithm ===")
  G = graph.copy()
  num_of_atoms = 0
  iteration = 0
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    num_of_atoms +=1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  graph = k_core_reduction(graph, len(k))
  if len(graph) == 0:
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  if len(graph) <= LIMIT:
    print("=== After K-core Reduction the Graph Size is Smaller than LIMIT ===")
    num_of_atoms += 1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  vertex_removal = {graph: []}
  subgraphs = [graph]
  while len(subgraphs) != 0:
    iteration += 1
    #print("Iteration number:", iteration)
    SG = subgraphs.pop()
    SG = remove_zero_degree_nodes(SG)
    #print("Current subgraph size:", len(SG))
    assert len(SG) != 0
    vcount = vertex_removal[SG]
    del vertex_removal[SG]
    SG_copy = SG.copy()
    # CHANGE GRAPH TO PY DATA OBJECT AND GET FEATURES
    new_py = graph_to_pyg_data(SG_copy)
    node_names = torch.unique(new_py.edge_index)
    node_names_list = node_names.tolist()
    num_nodes = new_py.num_nodes
    new_node_names = torch.arange(0, num_nodes)
    node_mapping = dict(zip(node_names.numpy(), new_node_names.numpy()))
    new_py.edge_index = torch.tensor([[node_mapping[edge[0].item()], node_mapping[edge[1].item()]] for edge in new_py.edge_index.t()])
    new_py.edge_index = new_py.edge_index.t()
    new_py.x = get_features(SG_copy)
    new_py.x = new_py.x.to(torch.float32)
    new_py = new_py.to(device)
    vertex = model_vertex_selection(model, new_py)
    vertex = list(SG_copy.nodes)[vertex]
    # REST OF THE ALGO
    SSG, SG = ch_partitioning(vertex, SG)
    SG = remove_zero_degree_nodes(SG) # BIG
    SSG = remove_zero_degree_nodes(SSG) # SMALL
    SG = k_core_reduction(SG, len(k)-len(vcount)) # 0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) # 1
    vertex_removal[SSG] = vcount+[vertex]
    vertex_removal[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removal[SSG])) == True
      if len(SSG)+len(vertex_removal[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removal[SSG]
      del vertex_removal[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removal[SG])) == True
      if len(SG)+len(vertex_removal[SG]) > len(k):
        k = list(SG.nodes())+vertex_removal[SG]
      del vertex_removal[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removal[SSG]
      assert is_clique(G.subgraph(SSG_lower)) == True
      if len(SSG_lower) > len(k):
        vcount = vertex_removal[SSG]
        del vertex_removal[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removal[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removal[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SSG))
            atom_sizes.append(len(SSG))
            num_of_atoms += 1
            del vertex_removal[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removal[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removal.keys()):
        sub_solution_SSG = vertex_removal[SSG]
        del vertex_removal[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removal[SG]
      assert is_clique(G.subgraph(SG_lower)) == True
      if len(SG_lower) > len(k):
        vcount = vertex_removal[SG]
        del vertex_removal[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removal[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removal[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SG))
            atom_sizes.append(len(SG))
            num_of_atoms += 1
            # sub_solution_SG = solver_function(SG)+vertex_removal[SG]
            del vertex_removal[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removal[SG]
    if len(SG) == 0:
      if SG in list(vertex_removal.keys()):
        sub_solution_SG = vertex_removal[SG]
        del vertex_removal[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
  assert len(vertex_removal) == 0
  print("=== Finished DBK Algorithm ===")
  end_time = time.time()
  elapsed_time = end_time - start_time
  return num_of_atoms, iteration, atom_sizes, elapsed_time

In [ ]:
import time
import torch

def DBK(graph, LIMIT=40):
  """
  INPUT:
    - "graph" must be a Networkx Undirected Graph
    - "LIMIT" is an integer describing the largest size of graph which solver_func can solve; all subgraph sizes solved will be less than or equal to LIMIT
    - "solver_function" takes a Networkx Graph, and outputs a list of nodes which are hopefully the Maximum Clique elements; it can be an approximate or exact solver function
  OUTPUT:
    - "k" is a list of graph nodes which form a clique in the input graph. If the solver is exact, then k is the Maximum Clique
  NOTES:
    - The central idea of using bounds is that we maintain a global lower bound on the Maximum Clique. Then, for each sub problem we calculate a fast upper bound.
      If any sub problem has an upper bound which is less than or equal to the global lower bound, we can remove that sub problem from consideration in the remaining iterations of the algorithm.
    - This algorithm does not necessarily enumerate all cliques nor all Maximum Cliques. In particular, it is designed to return a single maximum clique assuming the solver is exact.
      However, the algorithm could be modified to include all maximum cliques found from solving each sub-problem.
    - There are many assert statements in this function. These all serve as "sanity checks"; if any of them are tripped, something went wrong or an input was incorrect
  """
  start_time = time.time()
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting DBK Algorithm ===")
  G = graph.copy()
  num_of_atoms = 0
  iteration = 0
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    num_of_atoms +=1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  graph = k_core_reduction(graph, len(k))
  if len(graph) == 0:
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  if len(graph) <= LIMIT:
    print("=== After K-core Reduction the Graph Size is Smaller than LIMIT ===")
    num_of_atoms += 1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  vertex_removal = {graph: []}
  subgraphs = [graph]
  while len(subgraphs) != 0:
    iteration += 1
    SG = subgraphs.pop()
    SG = remove_zero_degree_nodes(SG)
    #print("Current subgraph size:", len(SG))
    assert len(SG) != 0
    vcount = vertex_removal[SG]
    del vertex_removal[SG]
    vertex = lowest_degree_vertex(SG)
    SSG, SG = ch_partitioning(vertex, SG)
    SG = remove_zero_degree_nodes(SG) # BIG
    SSG = remove_zero_degree_nodes(SSG) # SMALL
    SG = k_core_reduction(SG, len(k)-len(vcount)) # 0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) # 1
    vertex_removal[SSG] = vcount+[vertex]
    vertex_removal[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removal[SSG])) == True
      if len(SSG)+len(vertex_removal[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removal[SSG]
      del vertex_removal[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removal[SG])) == True
      if len(SG)+len(vertex_removal[SG]) > len(k):
        k = list(SG.nodes())+vertex_removal[SG]
      del vertex_removal[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removal[SSG]
      assert is_clique(G.subgraph(SSG_lower)) == True
      #print("SSG lower", len(SSG_lower))
      #print("lowerbound:", len(k))
      if len(SSG_lower) > len(k):
        vcount = vertex_removal[SSG]
        del vertex_removal[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removal[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removal[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SSG))
            atom_sizes.append(len(SSG))
            num_of_atoms += 1
            del vertex_removal[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removal[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removal.keys()):
        sub_solution_SSG = vertex_removal[SSG]
        del vertex_removal[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removal[SG]
      assert is_clique(G.subgraph(SG_lower)) == True
      #print("SG lower", len(SG_lower))
      #print("lowerbound:", len(k))
      if len(SG_lower) > len(k):
        vcount = vertex_removal[SG]
        del vertex_removal[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removal[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removal[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SG))
            atom_sizes.append(len(SG))
            num_of_atoms += 1
            # sub_solution_SG = solver_function(SG)+vertex_removal[SG]
            del vertex_removal[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removal[SG]
    if len(SG) == 0:
      if SG in list(vertex_removal.keys()):
        sub_solution_SG = vertex_removal[SG]
        del vertex_removal[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
  assert len(vertex_removal) == 0
  print("=== Finished DBK Algorithm ===")
  end_time = time.time()
  elapsed_time = end_time - start_time
  return num_of_atoms, iteration, atom_sizes, elapsed_time


def DBK_max(graph, LIMIT=40):
  """
  INPUT:
    - "graph" must be a Networkx Undirected Graph
    - "LIMIT" is an integer describing the largest size of graph which solver_func can solve; all subgraph sizes solved will be less than or equal to LIMIT
    - "solver_function" takes a Networkx Graph, and outputs a list of nodes which are hopefully the Maximum Clique elements; it can be an approximate or exact solver function
  OUTPUT:
    - "k" is a list of graph nodes which form a clique in the input graph. If the solver is exact, then k is the Maximum Clique
  NOTES:
    - The central idea of using bounds is that we maintain a global lower bound on the Maximum Clique. Then, for each sub problem we calculate a fast upper bound.
      If any sub problem has an upper bound which is less than or equal to the global lower bound, we can remove that sub problem from consideration in the remaining iterations of the algorithm.
    - This algorithm does not necessarily enumerate all cliques nor all Maximum Cliques. In particular, it is designed to return a single maximum clique assuming the solver is exact.
      However, the algorithm could be modified to include all maximum cliques found from solving each sub-problem.
    - There are many assert statements in this function. These all serve as "sanity checks"; if any of them are tripped, something went wrong or an input was incorrect
  """
  start_time = time.time()
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting DBK Algorithm ===")
  G = graph.copy()
  num_of_atoms = 0
  iteration = 0
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    num_of_atoms +=1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  graph = k_core_reduction(graph, len(k))
  if len(graph) == 0:
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  if len(graph) <= LIMIT:
    print("=== After K-core Reduction the Graph Size is Smaller than LIMIT ===")
    num_of_atoms += 1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  vertex_removal = {graph: []}
  subgraphs = [graph]
  while len(subgraphs) != 0:
    iteration += 1
    SG = subgraphs.pop()
    SG = remove_zero_degree_nodes(SG)
    #print("Current subgraph size:", len(SG))
    assert len(SG) != 0
    vcount = vertex_removal[SG]
    del vertex_removal[SG]
    vertex = highest_degree_vertex(SG)
    SSG, SG = ch_partitioning(vertex, SG)
    SG = remove_zero_degree_nodes(SG) # BIG
    SSG = remove_zero_degree_nodes(SSG) # SMALL
    SG = k_core_reduction(SG, len(k)-len(vcount)) # 0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) # 1
    vertex_removal[SSG] = vcount+[vertex]
    vertex_removal[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removal[SSG])) == True
      if len(SSG)+len(vertex_removal[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removal[SSG]
      del vertex_removal[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removal[SG])) == True
      if len(SG)+len(vertex_removal[SG]) > len(k):
        k = list(SG.nodes())+vertex_removal[SG]
      del vertex_removal[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removal[SSG]
      assert is_clique(G.subgraph(SSG_lower)) == True
      #print("SSG lower", len(SSG_lower))
      #print("lowerbound:", len(k))
      if len(SSG_lower) > len(k):
        vcount = vertex_removal[SSG]
        del vertex_removal[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removal[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removal[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SSG))
            atom_sizes.append(len(SSG))
            num_of_atoms += 1
            del vertex_removal[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removal[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removal.keys()):
        sub_solution_SSG = vertex_removal[SSG]
        del vertex_removal[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removal[SG]
      assert is_clique(G.subgraph(SG_lower)) == True
      #print("SG lower", len(SG_lower))
      #print("lowerbound:", len(k))
      if len(SG_lower) > len(k):
        vcount = vertex_removal[SG]
        del vertex_removal[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removal[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removal[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SG))
            atom_sizes.append(len(SG))
            num_of_atoms += 1
            # sub_solution_SG = solver_function(SG)+vertex_removal[SG]
            del vertex_removal[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removal[SG]
    if len(SG) == 0:
      if SG in list(vertex_removal.keys()):
        sub_solution_SG = vertex_removal[SG]
        del vertex_removal[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
  assert len(vertex_removal) == 0
  print("=== Finished DBK Algorithm ===")
  end_time = time.time()
  elapsed_time = end_time - start_time
  return num_of_atoms, iteration, atom_sizes, elapsed_time

def DBK_model(pygraph, rank_model, LIMIT=40):
  """
  INPUT:
    - "graph" must be a Networkx Undirected Graph
    - "LIMIT" is an integer describing the largest size of graph which solver_func can solve; all subgraph sizes solved will be less than or equal to LIMIT
    - "solver_function" takes a Networkx Graph, and outputs a list of nodes which are hopefully the Maximum Clique elements; it can be an approximate or exact solver function
  OUTPUT:
    - "k" is a list of graph nodes which form a clique in the input graph. If the solver is exact, then k is the Maximum Clique
  NOTES:
    - The central idea of using bounds is that we maintain a global lower bound on the Maximum Clique. Then, for each sub problem we calculate a fast upper bound.
      If any sub problem has an upper bound which is less than or equal to the global lower bound, we can remove that sub problem from consideration in the remaining iterations of the algorithm.
    - This algorithm does not necessarily enumerate all cliques nor all Maximum Cliques. In particular, it is designed to return a single maximum clique assuming the solver is exact.
      However, the algorithm could be modified to include all maximum cliques found from solving each sub-problem.
    - There are many assert statements in this function. These all serve as "sanity checks"; if any of them are tripped, something went wrong or an input was incorrect
  """
  start_time = time.time()
  features = pygraph.x
  print("all features:", features.shape)
  graph = to_networkx(pygraph)
  assert type(graph) is nx.Graph
  assert type(LIMIT) is int
  assert len(graph) != 0
  print("=== Starting DBK Algorithm ===")
  G = graph.copy()
  num_of_atoms = 0
  iteration = 0
  atom_sizes = []
  if len(graph) <= LIMIT:
    print("=== Input Graph Size is Smaller than LIMIT ===")
    num_of_atoms +=1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  print("Preprocessing...")
  graph = remove_zero_degree_nodes(graph)
  k = mc_lower_bound(graph)
  graph = k_core_reduction(graph, len(k))
  if len(graph) == 0:
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  if len(graph) <= LIMIT:
    print("=== After K-core Reduction the Graph Size is Smaller than LIMIT ===")
    num_of_atoms += 1
    print("=== Finished DBK Algorithm ===")
    end_time = time.time()
    elapsed_time = end_time - start_time
    return num_of_atoms, iteration, atom_sizes, elapsed_time
  vertex_removal = {graph: []}
  subgraphs = [graph]
  while len(subgraphs) != 0:
    iteration += 1
    #print("Iteration number:", iteration)
    SG = subgraphs.pop()
    SG = remove_zero_degree_nodes(SG)
    #print("Current subgraph size:", len(SG))
    assert len(SG) != 0
    vcount = vertex_removal[SG]
    del vertex_removal[SG]
    SG_copy = SG.copy()
    new_py = graph_to_pyg_data(SG_copy)
    # node names
    node_names = torch.unique(new_py.edge_index)
    node_names_list = node_names.tolist()
    num_nodes = new_py.num_nodes
    new_node_names = torch.arange(0, num_nodes)
    node_mapping = dict(zip(node_names.numpy(), new_node_names.numpy()))
    new_py.edge_index = torch.tensor([[node_mapping[edge[0].item()], node_mapping[edge[1].item()]] for edge in new_py.edge_index.t()])
    new_py.edge_index = new_py.edge_index.t()
    # node features
    new_features = features[node_names_list]
    new_py.x = new_features
    # Get the list of unique node indices (node names)
    vertex = model_vertex_selection(rank_model, new_py)
    vertex = list(SG_copy.nodes)[vertex]
    SSG, SG = ch_partitioning(vertex, SG)
    SG = remove_zero_degree_nodes(SG) # BIG
    SSG = remove_zero_degree_nodes(SSG) # SMALL
    SG = k_core_reduction(SG, len(k)-len(vcount)) # 0
    SSG = k_core_reduction(SSG, len(k)-len(vcount+[vertex])) # 1
    vertex_removal[SSG] = vcount+[vertex]
    vertex_removal[SG] = vcount
    #####################################################################################################
    if is_clique(G.subgraph(list(SSG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SSG.nodes())+vertex_removal[SSG])) == True
      if len(SSG)+len(vertex_removal[SSG]) > len(k):
        k = list(SSG.nodes())+vertex_removal[SSG]
      del vertex_removal[SSG]
      SSG = nx.Graph()
    if is_clique(G.subgraph(list(SG.nodes()))) == True:
      assert is_clique(G.subgraph(list(SG.nodes())+vertex_removal[SG])) == True
      if len(SG)+len(vertex_removal[SG]) > len(k):
        k = list(SG.nodes())+vertex_removal[SG]
      del vertex_removal[SG]
      SG = nx.Graph()
    #####################################################################################################
    if len(SSG) != 0:
      SSG_lower = mc_lower_bound(SSG)+vertex_removal[SSG]
      assert is_clique(G.subgraph(SSG_lower)) == True
      if len(SSG_lower) > len(k):
        vcount = vertex_removal[SSG]
        del vertex_removal[SSG]
        k = SSG_lower
        SSG = k_core_reduction(SSG, len(k)-len(vcount))
        SSG = remove_zero_degree_nodes(SSG)
        vertex_removal[SSG] = vcount
      if len(SSG) != 0:
        SSG_upper = mc_upper_bound(SSG)+len(vertex_removal[SSG])
        if SSG_upper > len(k):
          if len(SSG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SSG))
            atom_sizes.append(len(SSG))
            num_of_atoms += 1
            del vertex_removal[SSG]
          else:
            subgraphs.append(SSG)
        else:
          del vertex_removal[SSG]
    if len(SSG) == 0:
      if SSG in list(vertex_removal.keys()):
        sub_solution_SSG = vertex_removal[SSG]
        del vertex_removal[SSG]
        assert is_clique(G.subgraph(sub_solution_SSG)) == True
        if len(sub_solution_SSG) > len(k):
          k = sub_solution_SSG
    #####################################################################################################
    if len(SG) != 0:
      SG_lower = mc_lower_bound(SG)+vertex_removal[SG]
      assert is_clique(G.subgraph(SG_lower)) == True
      if len(SG_lower) > len(k):
        vcount = vertex_removal[SG]
        del vertex_removal[SG]
        k = SG_lower
        SG = k_core_reduction(SG, len(k)-len(vcount))
        SG = remove_zero_degree_nodes(SG)
        vertex_removal[SG] = vcount
      if len(SG) != 0:
        SG_upper = mc_upper_bound(SG)+len(vertex_removal[SG])
        if SG_upper > len(k):
          if len(SG) <= LIMIT:
            #print("=== Terminal Subgraph Found ===")
            #print("Size:", len(SG))
            atom_sizes.append(len(SG))
            num_of_atoms += 1
            # sub_solution_SG = solver_function(SG)+vertex_removal[SG]
            del vertex_removal[SG]
          else:
            subgraphs.append(SG)
        else:
          del vertex_removal[SG]
    if len(SG) == 0:
      if SG in list(vertex_removal.keys()):
        sub_solution_SG = vertex_removal[SG]
        del vertex_removal[SG]
        assert is_clique(G.subgraph(sub_solution_SG)) == True
        if len(sub_solution_SG) > len(k):
          k = sub_solution_SG
  assert len(vertex_removal) == 0
  print("=== Finished DBK Algorithm ===")
  end_time = time.time()
  elapsed_time = end_time - start_time
  return num_of_atoms, iteration, atom_sizes, elapsed_time


In [ ]:
# GCN Layer

class GCNLayer(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(GCNLayer, self).__init__()
        self.gcn = GCNConv(input_size, hidden_size)

    def forward(self, x, edge_index):
        return self.gcn(x, edge_index)


class RegressionModel(nn.Module):
    def __init__(self, input_size, gcn_hidden_size, dropout_prob=0):
        super(RegressionModel, self).__init__()
        # GCN layers
        self.gcn1 = GCNLayer(input_size, gcn_hidden_size)
        self.gcn2 = GCNLayer(gcn_hidden_size, gcn_hidden_size)
        self.gcn3 = GCNLayer(gcn_hidden_size, gcn_hidden_size)

        self.fc1 = nn.Linear(gcn_hidden_size, 128)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(128, 64)
        self.fc3 = nn.Linear(64, 32)
        self.fc4 = nn.Linear(32, 1)
        self.sigmoid = nn.Sigmoid()
        self.dropout = nn.Dropout(p=dropout_prob)
        self.batch_norm1 = nn.BatchNorm1d(128)
        self.batch_norm2 = nn.BatchNorm1d(64)
        self.batch_norm3 = nn.BatchNorm1d(32)

    def forward(self, data):
        x, edge_index = data.x, data.edge_index
        x = self.gcn1(x, edge_index)
        x = self.relu(x)
        x = self.gcn2(x, edge_index)
        x = self.relu(x)

        # MLP layers
        x = self.batch_norm1(self.fc1(x))
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm2(self.fc2(x))
        x = self.relu(x)
        x = self.dropout(x)
        x = self.batch_norm3(self.fc3(x))
        x = self.relu(x)
        x = self.fc4(x)
        x = self.sigmoid(x)
        return x


def train_model_regression(model, train_loader, val_loader, optimizer, criterion, batch_size, patience, epochs=10):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    train_losses = []
    val_losses = []
    base_path = '/content/drive/MyDrive/Colab Notebooks/Decomp/Models/regression'
    best_train_loss = float('inf')
    consecutive_no_improvement = 0
    for epoch in range(epochs):
        model.train()
        running_loss = 0.0
        start = 0
        train_epoch_loss = 0.0
        total_length = len(train_loader) // batch_size


        for i in range(start, len(train_loader), batch_size):
            optimizer.zero_grad()
            scores = torch.empty(0, dtype=torch.float32, requires_grad=True).to(device)
            labels = torch.empty(0, dtype=torch.float32, requires_grad=True).to(device)
            for j in range(start, start + batch_size):
                if j >= len(train_loader):
                    break
                graph = train_loader[j]
                temp = model(graph)
                scores = torch.cat([scores, temp], dim=0)
                temp = graph.y
                temp = temp.to(device)
                labels = torch.cat([labels, temp], dim=0)
            start += batch_size
            scores = scores.squeeze().to(device)
            loss = criterion(scores, labels)
            train_epoch_loss += loss.item()


            # Backward pass and optimization
            loss.backward()
            optimizer.step()

        average_train_loss = train_epoch_loss / total_length
        train_losses.append(average_train_loss)


        # Print progress
        if (epoch + 1) % 100 == 0:
            print(f'Epoch [{epoch+1}/{epochs}], '
                  f'Train Loss: {average_train_loss:.4f}')
        if train_losses:
            if min(train_losses) < best_train_loss:
                best_train_loss = min(train_losses)
                consecutive_no_improvement = 0
            else:
                consecutive_no_improvement += 1
            if consecutive_no_improvement >= patience:
                print(f'Early stopping at epoch {epoch+1} due to no improvement in training loss.')
                break
    # Plot the training and validation loss curves
    plt.plot(range(1, epoch+2), train_losses, label='Training Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend()
    plt.show()

In [ ]:

# Defining some data preprocessing functions
def graph_to_pyg_data(graph):
    edge_index = torch.tensor(list(graph.edges)).t().contiguous()
    x = torch.eye(graph.number_of_nodes())  # Node features, identity matrix in this example
    y = torch.tensor(list(nx.get_node_attributes(graph, 'label').values()))  # Node labels

    return Data(x=x, edge_index=edge_index, y=y)

def to_networkx(data):
    edge_index = data.edge_index.cpu().numpy()
    edge_attr = None
    if data.edge_attr is not None:
        edge_attr = data.edge_attr.cpu().numpy()

    G = nx.Graph()
    G.add_nodes_from(range(data.num_nodes))
    G.add_edges_from(edge_index.T)

    if edge_attr is not None:
        for i, (src, tgt) in enumerate(edge_index.T):
            G[src][tgt]['edge_attr'] = edge_attr[i]

    return G

    # min max normalize [0, 1]
def min_max_normalize(data, new_min=0, new_max=1):
    # Find the min and max values in the data
    min_val = min(data)
    max_val = max(data)
    if min_val == max_val:
      return [1 for _ in range(len(data))]
    normalized_data = [(x - min_val) / (max_val - min_val) * (new_max - new_min) + new_min for x in data]
    return normalized_data

# top k% of values are 1 rest are 0
def top_k(label_list, k):
    k = round(k*len(label_list))
    input_list = min_max_normalize(label_list)
    # Sort the list in descending order
    sorted_list = sorted(input_list, reverse=True)
    # Determine the threshold index
    threshold_index = min(k, len(sorted_list))
    # Set the first k elements to 1 and the rest to 0
    thresholded_list = [1 if i < threshold_index else 0 for i in range(len(sorted_list))]
    # Create a mapping from original indices to sorted indices
    index_mapping = {original: sorted_index for sorted_index, original in enumerate(sorted(range(len(input_list)), key=lambda x: input_list[x], reverse=True))}
    # Sort the thresholded list back to the original order
    thresholded_list_original_order = [thresholded_list[index_mapping[i]] for i in range(len(input_list))]

    return thresholded_list_original_order

# continuous [0,1]
def continuous(label_list, k):
  label_list = min_max_normalize(label_list)
  return label_list

# value above k are 1 rest are 0
def within_k(label_list, k):
  k = 1-k
  label_list = min_max_normalize(label_list)
  data = [1 if x > k else 0 for x in label_list]
  return data


def max_normalize_binary(label_list, k):
  k = 1-k
  # normalize data
  max_val = max(label_list)
  normalized_data = [(x / max_val) for x in label_list]
  data = [1 if x > k else 0 for x in normalized_data]
  return data

def max_normalize(label_list, k):
  # normalize data
  max_val = max(label_list)
  normalized_data = [(x / max_val) for x in label_list]
  return normalized_data


# TAKES IN A NETWORKX GRAPH AND OUTPUTS A TENSOR THAT IS THE NODE FEATURES
def get_features(nxgraph):
  # get features
  node_degrees = dict(nxgraph.degree())
  degree_centrality = nx.degree_centrality(nxgraph)
  betweenness_centrality = nx.betweenness_centrality(nxgraph)
  closeness_centrality = nx.closeness_centrality(nxgraph)
  eigenvector_centrality = nx.eigenvector_centrality(nxgraph, max_iter=500, tol=1.0e-3)
  pagerank_centrality = nx.pagerank(nxgraph, max_iter=500, tol=1.0e-3)
  harmonic_centrality = nx.harmonic_centrality(nxgraph)
  load_centrality = nx.load_centrality(nxgraph)
  clustering_coefficient = nx.clustering(nxgraph)
  # make it into an array
  features_array = np.array([
    list(node_degrees.values()),
    list(degree_centrality.values()),
    list(betweenness_centrality.values()),
    list(closeness_centrality.values()),
    list(eigenvector_centrality.values()),
    list(pagerank_centrality.values()),
    list(harmonic_centrality.values()),
    list(load_centrality.values()),
    list(clustering_coefficient.values())])
  features_array = features_array.T
  return torch.tensor(features_array)


In [ ]:
def model_vertex_selection(model, pygraph):
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    model.to(device)
    model.eval()
    best_node = None
    best_value = -9999
    # get labels
    outputs = model(pygraph)
    temp = outputs.detach().cpu()
    outputs = temp.numpy()
    best_value = -9999
    best_node = None
    for node, output_value in enumerate(outputs):
        if output_value > best_value:
            best_value = output_value
            best_node = node
    return best_node

In [ ]:
import numpy as np
loaded_dataset = torch.load('/content/NYSEgraph.pt')

FileNotFoundError: ignored

In [ ]:
import numpy as np
from tqdm.auto import tqdm


num_of_atoms_dbk = []
iteration_dbk = []
time_dbk = []

num_of_atoms_reg = []
iteration_reg = []
time_reg = []

num_of_atoms_max = []
iteration_max = []
time_max = []


for i in tqdm(range(len(loaded_dataset))):
    print(i+1)
    pygraph = loaded_dataset
    nxgraph = to_networkx(pygraph)
    nxgraph.remove_edges_from(nx.selfloop_edges(nxgraph))
    reg_model = RegressionModel(9, 256)
    num_of_atoms1, iteration1, atom_sizes1, time1 = DBK_model_instance(pygraph, reg_model, LIMIT=40)
    num_of_atoms_reg.append(num_of_atoms1)
    iteration_reg.append(iteration1)
    time_reg.append(time1)
    np.save('/content/NYSE_num_of_atoms_reg.npy', num_of_atoms_reg)
    np.save('/content/NYSE_iteration_reg.npy', iteration_reg)
    np.save('/content/NYSE_time_reg.npy', time_reg)
    #num_of_atoms2, iteration2, atom_sizes2, time2 = DBK(nxgraph, LIMIT=40)
    #num_of_atoms_dbk.append(num_of_atoms2)
    #iteration_dbk.append(iteration2)
    #time_dbk.append(time2)
    #num_of_atoms3, iteration3, atom_sizes3, time3 = DBK_max(nxgraph, LIMIT=40)
    #num_of_atoms_max.append(num_of_atoms3)
    #iteration_max.append(iteration3)
    #time_max.append(time3)

#np.save('/content/200_num_of_atoms_reg.npy', num_of_atoms_reg)
#np.save('/content/200_iteration_reg.npy', iteration_reg)
#np.save('/content/200_time_reg.npy', time_reg)
#np.save('/content/200_num_of_atoms_dbk.npy', num_of_atoms_dbk)
#np.save('/content/200_iteration_dbk.npy', iteration_dbk)
#np.save('/content/200_time_dbk.npy', time_dbk)
#np.save('/content/80_num_of_atoms_max.npy', num_of_atoms_max)
#np.save('/content/80_iteration_max.npy', iteration_max)
#np.save('/content/80_time_max.npy', time_max)

In [ ]:
loaded_dataset = torch.load('/content/drive/MyDrive/Colab Notebooks/Decomp/LabeledData/NetworkxGraphs/test_200nodes_100graphs.pt')

In [ ]:
densities = []
for graph in loaded_dataset:
    nxgraph = to_networkx(graph)
    nxgraph.remove_edges_from(nx.selfloop_edges(nxgraph))
    densities.append(nx.density(nxgraph))
print(densities)

[0.5048743718592965, 0.372964824120603, 0.6663316582914572, 0.20241206030150755, 0.5751256281407036, 0.44763819095477386, 0.14909547738693468, 0.3684924623115578, 0.20065326633165828, 0.5874874371859297, 0.41361809045226133, 0.5064321608040201, 0.16015075376884422, 0.125678391959799, 0.4948743718592965, 0.351608040201005, 0.4189949748743719, 0.33175879396984925, 0.6508542713567839, 0.3478894472361809, 0.4790954773869347, 0.2845226130653266, 0.5430653266331659, 0.4809547738693467, 0.33824120603015073, 0.4037185929648241, 0.624321608040201, 0.555929648241206, 0.4275376884422111, 0.345678391959799, 0.375427135678392, 0.30974874371859296, 0.5524623115577889, 0.21477386934673368, 0.30110552763819093, 0.21623115577889448, 0.2626633165829146, 0.6370854271356784, 0.3923115577889447, 0.4801005025125628, 0.37331658291457287, 0.20507537688442212, 0.6548743718592965, 0.13904522613065326, 0.3366331658291457, 0.6042211055276382, 0.1628643216080402, 0.5209045226130653, 0.5244221105527638, 0.132663316

In [25]:
np.save('/content/drive/MyDrive/Colab Notebooks/Decomp/Results/densities.npy', np.array(densities))